In [1]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/")

In [2]:
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

<Experiment: artifact_location='s3://mlflowexperiment-26/2', creation_time=1767599586800, experiment_id='2', last_update_time=1767599586800, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [4]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

ngram_ranges = [(1, 1), (1, 2), (1, 3)] 
max_features = 5000 

for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2026/01/05 17:32:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2/runs/7e5ed6a6d10245c4a2345b8330bf1090
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2


2026/01/05 17:34:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2/runs/d9660dcb92f5471fb1e9d30c68cd9a05
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2


2026/01/05 17:35:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2/runs/0806efce2a8244fca7f4875a08a9984a
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2


2026/01/05 17:38:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2/runs/d3bb925d4c9344a6b16eaf9f05ea6147
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2


2026/01/05 17:39:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2/runs/11ba605cf10b48d985ccd82b8d968d5d
🧪 View experiment at: http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/#/experiments/2
